In [1]:
import numpy as np
import re
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from __future__ import unicode_literals
from imblearn.combine import SMOTETomek
from math import *
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from IPython.core import display as ICD
from xgboost import XGBRegressor,XGBClassifier
from catboost import CatBoostRegressor,CatBoostClassifier
from lightgbm import LGBMClassifier,LGBMRegressor
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score,precision_score
from holidays import CountryHoliday
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV,LogisticRegression,ElasticNet
from sklearn.ensemble import BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import eli5
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report, f1_score,accuracy_score ,confusion_matrix
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import IsolationForest
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import Pipeline
import matplotlib.gridspec as gridspec
from sklearn.impute import KNNImputer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import seaborn as sns
import warnings
from warnings import simplefilter
import ast
import sklearn
simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (C:\Users\HP\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py)

In [2]:
train_df = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/train.csv.zip")
test_df = pd.read_csv("/kaggle/input/restaurant-revenue-prediction/test.csv.zip")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/restaurant-revenue-prediction/train.csv.zip'

In [3]:
train_df.head()

NameError: name 'train_df' is not defined

In [ ]:
train_df.sample(5)

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.head()

In [ ]:
train_df.drop("Id",axis=1,inplace = True)

In [ ]:
train_df["Open Date"] = pd.to_datetime(train_df["Open Date"])
train_df["Day"] = train_df["Open Date"].dt.day
train_df["Day_Name"] = train_df["Open Date"].dt.day_name()
train_df["Month"] = train_df["Open Date"].dt.month
train_df["Years"] = train_df["Open Date"].dt.year
train_df.drop("Open Date",axis=1,inplace=True)
#########
test_df["Open Date"] = pd.to_datetime(test_df["Open Date"])
test_df["Day"] = test_df["Open Date"].dt.day
test_df["Day_Name"] = test_df["Open Date"].dt.day_name()
test_df["Month"] = test_df["Open Date"].dt.month
test_df["Years"] = test_df["Open Date"].dt.year
test_df.drop("Open Date",axis=1,inplace=True)

In [ ]:
train_df["City"].unique()

In [ ]:
train_df["City Group"].unique()

In [ ]:
train_df["Type"].unique()

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(y= train_df["City"],palette="mako")
plt.xlabel("Number of Restaurants")
plt.tight_layout()

In [ ]:
city_df = train_df.groupby("City")["revenue"].agg(["mean","sum"]).reset_index()
fig , ax = plt.subplots(2,1,figsize = (20,15))
ax1 = sns.pointplot(x=city_df["City"],y=city_df["sum"],ax=ax[0])
ax1.tick_params(axis= "x" ,labelsize=15,labelrotation=75)
ax1.tick_params(axis= "y" ,labelsize=15)
ax1.set_xlabel("City", fontsize=15)
ax1.set_ylabel("Sum of Revenue", fontsize=15)
ax2 = sns.pointplot(x=city_df["City"],y=city_df["mean"],ax=ax[1],color = "r")
ax2.tick_params(axis= "x" ,labelsize=15,labelrotation=75)
ax2.tick_params(axis= "y" ,labelsize=15)
ax2.set_xlabel("City", fontsize=15)
ax2.set_ylabel("Mean of Revenue", fontsize=15)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
group_dict = dict(train_df["City Group"].value_counts())
plt.pie(group_dict.values(),labels=group_dict.keys(),wedgeprops={"edgecolor":"black"},autopct="%1.1f%%")
plt.tight_layout()

In [ ]:
cityg_df = train_df.groupby("City Group")["revenue"].agg(["mean","sum"]).reset_index()
fig , ax = plt.subplots(2,1,figsize = (18,12))
ax1 = sns.barplot(x=cityg_df["City Group"],y=cityg_df["sum"],ax=ax[0])
ax1.set_ylabel("Sum of Revenue")
ax2 = sns.barplot(x=cityg_df["City Group"],y=cityg_df["mean"],ax=ax[1])
ax2.set_ylabel("Mean of Revenue")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
type_dict = dict(train_df["Type"].value_counts())
ex = [0,0.03,0.7]
plt.pie(type_dict.values(),labels=type_dict.keys(),explode = ex,wedgeprops={"edgecolor":"black"},autopct="%1.1f%%")
plt.tight_layout()

In [ ]:
type_df = train_df.groupby("Type")["revenue"].agg(["mean","sum"]).reset_index()
fig , ax = plt.subplots(2,1,figsize = (18,12))
ax1 = sns.barplot(x=type_df["Type"],y=type_df["sum"],ax=ax[0])
ax1.set_ylabel("Sum of Revenue")
ax2 = sns.barplot(x=type_df["Type"],y=type_df["mean"],ax=ax[1])
ax2.set_ylabel("Mean of Revenue")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x= train_df["Day"],palette="flare")
plt.ylabel("Number of Restaurants")
plt.tight_layout()

In [ ]:
day_df = train_df.groupby("Day")["revenue"].agg(["mean","sum"]).reset_index()
fig , ax = plt.subplots(2,1,figsize = (18,12))
ax1 = sns.barplot(x=day_df["Day"],y=day_df["sum"],ax=ax[0],palette = "magma")
ax1.set_ylabel("Sum of Revenue")
ax2 = sns.barplot(x=day_df["Day"],y=day_df["mean"],ax=ax[1],palette = "magma")
ax2.set_ylabel("Mean of Revenue")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x= train_df["Day_Name"])
plt.ylabel("Number of Restaurants")
plt.tight_layout()

In [ ]:
day_Name_df = train_df.groupby("Day_Name")["revenue"].agg(["mean","sum"]).reset_index()
fig , ax = plt.subplots(2,1,figsize = (18,12))
ax1 = sns.barplot(x=day_Name_df["Day_Name"],y=day_Name_df["sum"],ax=ax[0])
ax1.set_ylabel("Sum of Revenue")
ax2 = sns.barplot(x=day_Name_df["Day_Name"],y=day_Name_df["mean"],ax=ax[1])
ax2.set_ylabel("Mean of Revenue")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x= train_df["Month"],palette="summer")
plt.ylabel("Number of Restaurants")
plt.tight_layout()

In [ ]:
Month_df = train_df.groupby("Month")["revenue"].agg(["mean","sum"]).reset_index()
fig , ax = plt.subplots(2,1,figsize = (18,12))
ax1 = sns.barplot(x=Month_df["Month"],y=Month_df["sum"],ax=ax[0],palette = "rocket")
ax1.set_ylabel("Sum of Revenue")
ax2 = sns.barplot(x=Month_df["Month"],y=Month_df["mean"],ax=ax[1],palette = "rocket")
ax2.set_ylabel("Mean of Revenue")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x= train_df["Years"],palette="inferno")
plt.ylabel("Number of Restaurants")
plt.tight_layout()

In [ ]:
Years_df = train_df.groupby("Years")["revenue"].agg(["mean","sum"]).reset_index()
fig , ax = plt.subplots(2,1,figsize = (18,12))
ax1 = sns.barplot(x=Years_df["Years"],y=Years_df["sum"],ax=ax[0],palette = "inferno")
ax1.set_ylabel("Sum of Revenue")
ax2 = sns.barplot(x=Years_df["Years"],y=Years_df["mean"],ax=ax[1],palette = "inferno")
ax2.set_ylabel("Mean of Revenue")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,7))
sns.distplot(x= train_df["revenue"])
plt.tight_layout()

In [ ]:
fig , ax = plt.subplots(7,6,figsize = (45,29))
r = 0
c = 0
for column in train_df.columns[4:41]:
    if r % 2 == 0 :
        if (c == 0) or (c == 2) or (c==4):
            pal = "viridis"
        else:
            pal = "flare"
    else:
        if (c == 0) or (c == 2) or (c==4) :
            pal = "flare"
        else:
            pal = "viridis"
    ax1 = sns.countplot(x=train_df[column],ax=ax[r,c],palette=pal)
    ax1.tick_params(axis= "x" ,labelsize=23)
    ax1.tick_params(axis= "y" ,labelsize=23)
    ax1.set_title(f"({column}) Count",size=23)
    ax1.set_xlabel(None, fontsize=23)
    ax1.set_ylabel(None, fontsize=23)
    c += 1
    if c > 5:
        r+=1
        c=0
    plt.tight_layout()

In [ ]:
train_df.head()

In [ ]:
class City_Group(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        X["City Group"] = X["City Group"].apply(lambda x: 1 if x == "Big Cities" else 0)
        return X
class Type(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        dummies = pd.get_dummies(X["Type"])
        X = pd.concat([X,dummies],axis=1)
        X.drop("Type",axis=1,inplace=True)
        return X
class City(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        dummies = pd.get_dummies(X["City"])
        X = pd.concat([X,dummies],axis=1)
        X.drop("City",axis=1,inplace=True)
        return X

In [ ]:
pip = Pipeline([
    ("City_Group",City_Group()),
    ("Type",Type()),
    ("City",City())
])

In [ ]:
train_df = pip.fit_transform(train_df).drop("Day_Name",axis=1)

In [ ]:
train_df.head()

In [ ]:
plt.figure(figsize=(12,7))
sns.heatmap(train_df.corr())
plt.tight_layout()

In [ ]:
corr = train_df.corr()["revenue"].abs()
sorted_corr = corr.sort_values()
num_to_drop = int(0.75* len(train_df.columns))
cols_to_drop = sorted_corr.iloc[:num_to_drop].index
df_corr = train_df.drop(cols_to_drop,axis=1)

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_corr.corr(),annot=True)
plt.tight_layout()

In [ ]:
train_df["Çorum"] = 0
train_df["Çankırı"] = 0
train_df["Çanakkale"] = 0
train_df["Zonguldak"] = 0
train_df["Yalova"] = 0
train_df["Tanımsız"] = 0
train_df["Sivas"] = 0
train_df["Siirt"] = 0
train_df["Rize"] = 0
train_df["Ordu"] = 0
train_df["Niğde"] = 0
train_df["Nevşehir"] = 0
train_df["Mersin"] = 0
train_df["Mardin"] = 0
train_df["Manisa"] = 0
train_df["Malatya"] = 0
train_df["MB"] = 0
train_df["Kırşehir"] = 0
train_df["Kırıkkale"] = 0
train_df["Kars"] = 0
train_df["Kahramanmaraş"] = 0
train_df["Hatay"] = 0
train_df["Giresun"] = 0
train_df["Erzurum"] = 0
train_df["Erzincan"] = 0
train_df["Düzce"] = 0
train_df["Bilecik"] = 0
train_df["Batman"] = 0
train_df["Artvin"] = 0
train_df["Aksaray"] = 0

In [ ]:
train_df

In [ ]:
X = train_df.drop("revenue",axis=1)
y = train_df["revenue"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

In [ ]:
def RSE(y_true, y_predicted):
    y_true = np.array(y_true)
    y_predicted = np.array(y_predicted)
    RSS = np.sum(np.square(y_true - y_predicted))

    rse = sqrt(RSS / (len(y_true) - 2))
    return rse

In [ ]:
models=[LinearRegression(),RandomForestRegressor(n_estimators=42,n_jobs=-1),AdaBoostRegressor(),KNeighborsRegressor(),DecisionTreeRegressor(),GradientBoostingRegressor(),XGBRegressor(),LGBMRegressor(),CatBoostRegressor(),Lasso(),Ridge()]
model_names=["LinearRegression","RandomForestRegressor","AdaBoostRegressor","KNeighborsRegressor","DecisionTreeRegressor","GradientBoostingRegressor","XGBRegressor","LGBMRegressor","CatBoostRegressor","Lasso","Ridge"]
MSE=[]
r_2_test=[]
r_2_train = []
RSE_ = []
accuracy = []
RMSE = []
MAE = []
for model in range (len(models)):
    model=models[model]
    model.fit(X_train,y_train)
    y_pre=model.predict(X_test)
    C = cross_validate(model,X,y,cv=5,scoring=("neg_mean_squared_error","neg_root_mean_squared_error","neg_mean_absolute_error","r2"),return_train_score=True)
    MSE.append((C["test_neg_mean_squared_error"] * -1).mean())
    MAE.append((C["test_neg_mean_absolute_error"] * -1).mean())
    r_2_train.append(C["train_r2"].mean())
    r_2_test.append(C["test_r2"].mean())
    RSE_.append(round(RSE(y_test,y_pre),5))
    accuracy.append((model.score(X_test,y_test))*100)
    RMSE.append((C["test_neg_root_mean_squared_error"] * -1).mean())
model_report=pd.DataFrame({'Modelling Name':model_names,"Train R2":r_2_train,'MSE':MSE,"Test R2":r_2_test,"RSE":RSE_,"Accuracy":accuracy,"RMSE":RMSE,"MAE":MAE})

In [ ]:
model_report

In [ ]:
model = RandomForestRegressor()

In [ ]:
param_grid = {
    "n_estimators" : [100,150,200,250,300],
    "max_depth" : [10,15,20,None],
    "n_jobs" : [1,-1,None],
    "verbose" : [0,1,2,-1]
}

In [ ]:
gr = GridSearchCV(model,param_grid=param_grid,cv=3)

In [ ]:
gr.fit(X,y)

In [ ]:
gr.best_params_

In [ ]:
gr.best_estimator_

In [ ]:
model = gr.best_estimator_

In [ ]:
test_df.head()

In [ ]:
ids = test_df["Id"]
test_df.drop("Id",axis=1,inplace=True)

In [ ]:
test_df = pip.fit_transform(test_df).drop("Day_Name",axis=1)

In [ ]:
test_df["Amasya"] = 0
test_df["Elazığ"] = 0
test_df["Karabük"] = 0
test_df["Kastamonu"] = 0
test_df["Osmaniye"] = 0
test_df["Tokat"] = 0

In [ ]:
test_df = test_df[X.columns]

In [ ]:
Prediction = model.predict(test_df)

In [ ]:
Predictions = pd.DataFrame({"Id":ids,"Prediction":Prediction})

In [ ]:
Predictions

In [ ]:
Predictions.to_csv("submission.csv",index=False)